<a href="https://colab.research.google.com/github/Juuus/srgbm-mfpt/blob/main/srgbm_mfpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **srGBM-MFPT ESTIMATIONS**

How much time does it take for individual workers to improve their income status? This is the bedrock question that lies beneath the existence of the American dream. By employing ideas and techniques from statistical mechanics, we can provide a disaggregated view on a worker's income timeline. Here you can input your own data and calculate estimates for mean-first-passage estimates for the time needed for an individual worker to change their income $x_0$ to a target income $y$.

 You can choose whether you want to provide the data in csv/xlsx format or input it directly in a table. For more explanations about the data needed for these calculations we refer to [REF TO OUR WORK].



In [201]:
#@title
# Import packages
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import random
import statsmodels.api as sm
import ipywidgets as widgets
import scipy
from scipy.optimize import minimize
import scipy.stats as st
from scipy.stats import gmean
from scipy import signal

# Define global variables



# Importing the data
First, we need to calculate the parameters that drive the income in your economy. For this, you will need to provide two datasets:

# Dataset 1.

Insert a table as an xlsx/csv document that provides how the share of the income owned by the top 1% has evolved over time. The data should have two columns. The first column should indicate the year, and it should be called 'year'. The second should give the value of the share of the income owned by the top 1% and should be called share. You must include at least two data points. As an example, see the document provided here.

In [99]:
#@title
 !rm *
from google.colab import files
upload1 = widgets.FileUpload(multiple = False, description='Upload Share 1%',accept='.xlsx,csv')
display(upload1)

rm: cannot remove 'sample_data': Is a directory


FileUpload(value={}, accept='.xlsx,csv', description='Upload Share 1%')

In [100]:
#@title
filename1 = next(iter(upload1.value))
content1 = upload.value[filename1]['content']
with open(filename1, 'wb') as f: 
  f.write(content1)
if filename1.endswith('.xlsx'):
  data_share = pd.read_excel(filename,sheet_name='Sheet1')
elif filename1.endswith('.csv'):  
  data_share = pd.read_csv(filename)
 
data_sh = data_share[data_share.columns[1]].values

# Dataset 2. 

Insert a table as an xlsx/csv document that provides how the resetting rate has evolved over time. A good approximation for the resetting rate is the share of workers who left/lost their job in a calendar year. The first column of the table should indicate the year, and should be called 'year'. The second should give the value of the resetting rate and should be called 'rate. You must include data for the same years as in Step 1. As an example, see this document [downloads a csv for USA top 1 share].

In [101]:
#@title
upload2 = widgets.FileUpload(multiple = False, description='Upload Reset rate%',accept='.xlsx,csv')
display(upload2)

FileUpload(value={}, accept='.xlsx,csv', description='Upload Reset rate%')

In [102]:
#@title
filename2 = next(iter(upload2.value))
content2 = upload2.value[filename2]['content']
with open(filename2, 'wb') as f: 
  f.write(content2)
if filename2.endswith('.xlsx'):
  rs_ = pd.read_excel(filename2,sheet_name='Sheet1')
elif filename2.endswith('.csv'):  
  rs_ = pd.read_csv(filename2)
 
rs = rs_[rs_.columns[1]].values

start_year = min(rs_['year'])
end_year = max(rs_['year'])
mu0 = 0.06 # initial mu
sig0 = np.sqrt(0.02219277) #initial sigma

# Estimating the srGBM parameters
Now we are equipped to calculate the income dynamics parameters of your economy. 

In [119]:
#@title
def estimate_srgbm_parameters(iterations):
  

  
  for iteration in range(iterations):
    print("Percentage completed: " + str(round(100*(1+iteration)/iterations,2)) + "%")   
    min_location = []
    min_pred = []
    min_sh = []
    
    random.seed(10)
    t = len(data_sh)
    people = 10000
    dt = 1
    m = 10
    trajs = np.ones([m, m, t+1, people])
    for real in range(0,t-1,1):
      sh = []
      sqerror = []
      pred = []
    
      if real == 0:
        trajs[:,0,:] = np.ones([m,t+1,people]) # da se smeni soglasno diskusija so Petar_
        mus = np.linspace(mu0,mu0,m)
        sigmas = np.linspace(sig0,sig0,m)
      else:   
        mus = np.linspace(0.001,0.15,m)
        sigmas = np.linspace(0.20,0.70,m)
        
      lista = {}
      
      for muiter, mu in enumerate(mus):
        for siter, sigma in enumerate(sigmas):
          choice_ = [[0,1],[1-rs[real+1]*dt, rs[real+1]*dt]]
          prob = np.random.choice(a=choice_[0], p=choice_[1], size=people)
          noise = np.random.randn(1,people)
    
          trajs[muiter,siter, real + 1, np.argwhere(prob == 0)] = trajs[muiter,siter, real, np.argwhere(prob == 0)] * (1 + mu * dt + (sigma * np.sqrt(dt)) * noise[0,np.argwhere(prob == 0)])
          trajs[muiter,siter, real + 1, np.argwhere(prob == 1)] = np.min(trajs[muiter,siter, 0, :])
        
          check = trajs[muiter,siter, real + 1, :]
          trajs[muiter,siter, real + 1, np.where(check<0)] = np.min(trajs[muiter,siter, 0, :]) #trajs[ri, 0, np.where(check<0)]
    
          trajs[muiter,siter, real+1, :].sort()
        
          share = np.sum(trajs[muiter, siter, real+1,:][-100:])/np.sum(trajs[muiter, siter, real+1,:])
     
          sqerror.append((share-data_sh[real+1])**2)
    
          sh.append(share)
              
          lista[muiter,siter] = mu, sigma   
              
      min_loc = sqerror.index(min(sqerror))
      min_location.append(list(lista.values())[min_loc])
      min_sh.append(sh[min_loc])
    
    fitted_mu[:, iteration] = np.array(pd.DataFrame(min_location)[0])
    fitted_sigma[:, iteration] = np.array(pd.DataFrame(min_location)[1])
    fitted_ddps[:, iteration] = min_sh

iterations = 25;
fitted_mu = np.ones([len(data_sh)-1, iterations])
fitted_sigma = np.ones([len(data_sh)-1, iterations])
fitted_ddps = np.ones([len(data_sh)-1, iterations])
button = widgets.Button(description="Estimate parameters")
display(button)

def on_button_clicked(b):
    estimate_srgbm_parameters(iterations)


button.on_click(on_button_clicked)

Button(description='Estimate parameters', style=ButtonStyle())

Percentage completed: 4.0%
Percentage completed: 8.0%
Percentage completed: 12.0%
Percentage completed: 16.0%
Percentage completed: 20.0%
Percentage completed: 24.0%
Percentage completed: 28.0%
Percentage completed: 32.0%
Percentage completed: 36.0%
Percentage completed: 40.0%
Percentage completed: 44.0%
Percentage completed: 48.0%
Percentage completed: 52.0%
Percentage completed: 56.0%
Percentage completed: 60.0%
Percentage completed: 64.0%
Percentage completed: 68.0%
Percentage completed: 72.0%
Percentage completed: 76.0%
Percentage completed: 80.0%
Percentage completed: 84.0%
Percentage completed: 88.0%
Percentage completed: 92.0%
Percentage completed: 96.0%
Percentage completed: 100.0%
Percentage completed: 4.0%
Percentage completed: 8.0%
Percentage completed: 12.0%
Percentage completed: 16.0%
Percentage completed: 20.0%
Percentage completed: 24.0%
Percentage completed: 28.0%
Percentage completed: 32.0%
Percentage completed: 36.0%
Percentage completed: 40.0%
Percentage completed: 4

# Calculate the srGBM MFPT
Now that you have calculated the parameters of your economy, you can choose the starting income ($x_0$), target income ($y$), and the value to which incomes are reset ($x_r$), and estimate the srGBM-MFPT. 

The results are coupled with 95% confidence intervals.

In [214]:
#@title
layout = widgets.Layout(width='auto', height='40px') #set width and height

x0_temp = widgets.FloatText(
    value=1000,
    description='Starting Income:',
    disabled=False,
    style= {'description_width': 'initial'}
)
display(x0_temp)

y_temp = widgets.FloatText(
    value=10000,
    description='Target Income:   ',
    disabled=False,
    style= {'description_width': 'initial'}
)
display(y_temp)

xr_temp = widgets.FloatText(
    value=100,
    description='Resetting Income:',
    disabled=False,
    style= {'description_width': 'initial'}
)
display(xr_temp)

FloatText(value=1000.0, description='Starting Income:', style=DescriptionStyle(description_width='initial'))

FloatText(value=10000.0, description='Target Income:   ', style=DescriptionStyle(description_width='initial'))

FloatText(value=100.0, description='Resetting Income:', style=DescriptionStyle(description_width='initial'))

In [217]:
#@title

xr = xr_temp.value #resetting income
x0 = x0_temp.value #initial income
ub = y_temp.value #target income
start = start_year
end = end_year

year1 = np.arange(start,end+1,1).reshape(end-start+1,1) 
#%% Analytic MFPT function

mfpt = np.zeros((len(year1),iterations))
mean_mfpt = np.zeros((len(year1),1))
ub_mfpt = np.zeros((len(year1),1))
lb_mfpt = np.zeros((len(year1),1))


def mfpt_analytic(r, params):
    
    mu, sigma, x0, ub, xr = params
    
    q = (np.sqrt((sigma**2-2*mu)**2 + 8*r*sigma**2) + (sigma**2-2*mu)) / (2*sigma**2)
    
    Tx0 = (x0/ub)**q
    Tr = (xr/ub)**q
    
    mean_Tr = (1-Tx0) / (r*Tr)
    
    return mean_Tr
# Read the fitted parameters

def plot_results():
  std_errors = 2



  for i in range(0,len(year1)-1):
    r_est = rs[i]
    for j in range(0,iterations-1):
      mu_est = fitted_mu[i,j]
      sigma_est = fitted_sigma[i,j]
        
      params_baseline = np.array([mu_est,sigma_est,x0,ub,xr])
      mfpt[i,j] = mfpt_analytic(r_est,params_baseline)

  years = rs_['year'].values
  mean_mfpt = np.mean(mfpt, axis=1)
  confidence_interval = 2 * np.std(mfpt, axis=1)/np.sqrt(iterations)
  ub_mfpt = mean_mfpt + confidence_interval
  lb_mfpt = mean_mfpt - confidence_interval
  mean_mfpt = signal.savgol_filter(mean_mfpt,5,2);
  ub_mfpt = signal.savgol_filter(ub_mfpt,5,2);
  lb_mfpt = signal.savgol_filter(lb_mfpt,5,2);  
  mean_mfpt[np.where(mean_mfpt<=0)] = 0
  ub_mfpt[np.where(ub_mfpt<=0)] = 0
  lb_mfpt[np.where(lb_mfpt<=0)] = 0

  fig = go.Figure([
      go.Scatter(
          name='Estimate',
          x = years,
          y= mean_mfpt, 
          mode='lines+markers',  
          line=dict(color='royalblue', width=2),  
          marker=dict(size=10,color='royalblue',symbol='circle'),
          showlegend=False
    ),
    go.Scatter(
        name='Upper Bound',
        x = years,
        y = ub_mfpt,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x = years,
        y = lb_mfpt,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
  ])
  fig.update_layout(
    yaxis = dict(title="MFPT in (years)",
                 showgrid = True,gridcolor = "black", showline = False, showticklabels = True),
    xaxis = dict(title="Years",zeroline = True, showline = True,showticklabels = True,tickangle=0, showgrid = True),
    hovermode="x",
   # paper_bgcolor='rgba(255,255,255,0)',
    plot_bgcolor='rgba(255,255,255,0)'
  )
  fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
  fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
  fig.show()

button2 = widgets.Button(description="Visualize results")
display(button2)
def on_button_clicked(b):
    plot_results()


button2.on_click(on_button_clicked)





Button(description='Visualize results', style=ButtonStyle())

You can download your results in csv format here:

In [223]:
button3 = widgets.Button(description="Download results")
display(button3)
def on_button_clicked(b):
  years = rs_['year'].values
  mean_mfpt = np.mean(mfpt, axis=1)
  confidence_interval = 2 * np.std(mfpt, axis=1)/np.sqrt(iterations)
  ub_mfpt = mean_mfpt + confidence_interval
  lb_mfpt = mean_mfpt - confidence_interval
  mean_mfpt = signal.savgol_filter(mean_mfpt,5,2);
  ub_mfpt = signal.savgol_filter(mean_mfpt,5,2);
  lb_mfpt = signal.savgol_filter(mean_mfpt,5,2);
  mean_mfpt[np.where(mean_mfpt<=0)] = 0
  ub_mfpt[np.where(ub_mfpt<=0)] = 0
  lb_mfpt[np.where(lb_mfpt<=0)] = 0


  dikt = {"year": list(years), "MFPT" : list (mean_mfpt), "MFPT UB" : list (ub_mfpt), "MFPT LB":list (lb_mfpt)}
  mfpt_results = pd.DataFrame(dikt)
  mfpt_results.to_csv('mfpt_results.csv',index=False)
  files.download('mfpt_results.csv')


button3.on_click(on_button_clicked)

Button(description='Download results', style=ButtonStyle())